# Metacritic - Web scraping

In [167]:
import requests
import urllib2
from bs4 import BeautifulSoup
from string import ascii_lowercase
import pandas as pd

In [168]:
def process_page(url):
    print url
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(url,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page,"lxml")
    obj = soup.find("ol", class_="list_products list_product_condensed")
    if not obj:
        return None
    obj_item = obj.find_all("div","basic_stat product_title")
    if not obj_item:
        return None
    df = pd.DataFrame()
    headers = ['movie_title', 'movie_score', 'release_date']
    for tag in obj_item:
        title = tag.getText().encode('ascii','ignore')
        if title:
            title = title.replace("\n","").strip(" ")
        next_tag = tag.findNext("div", class_="basic_stat product_score brief_metascore")
        if next_tag:
            score = next_tag.getText().encode('ascii','ignore')
            if score:
                score = score.replace("\n","").strip(" ")
        next_tag = next_tag.findNext("li", class_="stat release_date full_release_date")
        if next_tag:
            date = next_tag.getText().encode('ascii','ignore')
            if date:
                date = date.replace("\n","").strip(" ")
                date = date.split(":")[-1]
        # Needs to be adjusted for the fact that sometimes release date is missing
        # Now: if missing, returns the release date of the next movie in row!!
        row = pd.DataFrame([[title, score, date]], columns=headers)
        df = df.append(row, ignore_index=True)
    page.close()
    return df

def process_metacritic(main_url):
    df = process_page(main_url)
    nums = "?page=1"
    df_critic = process_page(main_url+nums)
    df = df.append(df_critic, ignore_index=True)
    df.to_pickle('df_critic.pickle')
    
    num_dict = { 'a' : 6, 'b' : 5, 'c' : 4, 'd' : 4, 'e' : 2, 'f' : 3, 'g' : 3,
                 'h' : 4, 'i' : 3, 'j' : 2, 'k' : 2, 'l' : 4, 'm' : 5, 'n' : 2,
                 'o' : 2, 'p' : 3, 'q' : 1, 'r' : 3, 's' : 7, 't' : 19, 'u': 1,
                 'v' : 1, 'w' : 3, 'x' : 1, 'y' : 1, 'z' : 1
        
    }
    
    for c in ascii_lowercase:
        page = main_url + "/" + c 
        df_critic = process_page(page)
        df = pd.read_pickle('df_critic.pickle')
        df = df.append(df_critic, ignore_index=True)
        df.to_pickle('df_critic.pickle')
        for num in range(1,num_dict[c]):
            page = main_url + "/" + c + "?page=" + str(num)
            df = pd.read_pickle('df_critic.pickle')
            df_critic = process_page(page)
            if df_critic is not None:
                df = df.append(df_critic, ignore_index=True)
                df.to_pickle('df_critic.pickle')
            
    df.to_csv("/home/lucia/lucia/metis/02-luther/data/metacritic.csv", index = False)
    

In [169]:
main_url= "http://www.metacritic.com/browse/movies/title/dvd"
process_metacritic(main_url)

http://www.metacritic.com/browse/movies/title/dvd
http://www.metacritic.com/browse/movies/title/dvd?page=1
http://www.metacritic.com/browse/movies/title/dvd/a
http://www.metacritic.com/browse/movies/title/dvd/a?page=1
http://www.metacritic.com/browse/movies/title/dvd/a?page=2
http://www.metacritic.com/browse/movies/title/dvd/a?page=3
http://www.metacritic.com/browse/movies/title/dvd/a?page=4
http://www.metacritic.com/browse/movies/title/dvd/a?page=5
http://www.metacritic.com/browse/movies/title/dvd/b
http://www.metacritic.com/browse/movies/title/dvd/b?page=1
http://www.metacritic.com/browse/movies/title/dvd/b?page=2
http://www.metacritic.com/browse/movies/title/dvd/b?page=3
http://www.metacritic.com/browse/movies/title/dvd/b?page=4
http://www.metacritic.com/browse/movies/title/dvd/c
http://www.metacritic.com/browse/movies/title/dvd/c?page=1
http://www.metacritic.com/browse/movies/title/dvd/c?page=2
http://www.metacritic.com/browse/movies/title/dvd/c?page=3
http://www.metacritic.com/bro